In [12]:
import pandas as pd
import numpy as np
import os

def contar_pares_integrados_csv(arquivo_csv, alpha=0.05):
    """
    Conta o número de pares com p-valor menor que alpha em um arquivo CSV.

    Args:
        arquivo_csv (str): Caminho para o arquivo CSV contendo a matriz de p-values.
                           Espera-se que a primeira coluna seja o índice dos ativos
                           e as colunas também representem os ativos.
        alpha (float): Nível de significância (default: 0.05).

    Returns:
        int: Número de pares integrados. Retorna None em caso de erro na leitura.
    """
    try:
        df_pvalues = pd.read_csv(arquivo_csv, header=None)
        if not isinstance(df_pvalues, pd.DataFrame) or df_pvalues.empty:
            print(f"Erro ao ler ou DataFrame vazio: {arquivo_csv}")
            return None

        n = df_pvalues.shape[0]
        if n != len(df_pvalues.columns):
            print(f"Erro: DataFrame não é quadrado em {arquivo_csv}")
            return None

        contador = 0
        colunas = df_pvalues.columns.tolist()
        index_nomes = df_pvalues.index.tolist()

        if colunas != index_nomes:
            print(f"Aviso: Nomes de colunas e índice não correspondem em {arquivo_csv}. Assumindo ordem consistente.")

        for i in range(n):
            for j in range(i + 1, n):
                try:
                    p_value = df_pvalues.loc[index_nomes[i], colunas[j]]
                    if isinstance(p_value, pd.Series):
                        print(f"Erro: p_value para ({index_nomes[i]}, {colunas[j]}) é uma Series em {arquivo_csv}: {p_value.values}")
                        continue  # Ignora este par se o p-value não for escalar
                    elif isinstance(p_value, (int, float)):
                        if p_value < alpha:
                            contador += 1
                    else:
                        print(f"Aviso: p_value para ({index_nomes[i]}, {colunas[j]}) não é numérico em {arquivo_csv}: {p_value}")
                except KeyError as e:
                    print(f"Erro de KeyError ao acessar ({index_nomes[i]}, {colunas[j]}) em {arquivo_csv}: {e}")
                    continue

        return contador

    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado: {arquivo_csv}")
        return None
    except Exception as e:
        print(f"Erro inesperado ao processar {arquivo_csv}: {e}")
        return None

def analisar_pares_integrados_por_semestre_df_corrigido(pasta_csv, alpha=0.05):
    """
    Analisa todos os arquivos CSV em uma pasta, contando pares integrados em cada um,
    e retorna um DataFrame com o semestre como índice e a contagem de pares como coluna.

    Args:
        pasta_csv (str): Caminho para a pasta contendo os arquivos CSV.
        alpha (float): Nível de significância (default: 0.05).

    Returns:
        pd.DataFrame: DataFrame com o índice 'Semestre' e a coluna 'NumParesIntegrados'.
    """
    resultados = {}
    arquivos_csv = sorted([f for f in os.listdir(pasta_csv) if f.startswith('pvalues')])
    for i, arquivo in enumerate(arquivos_csv):
        caminho_completo = os.path.join(pasta_csv, arquivo)
        num_pares = contar_pares_integrados_csv(caminho_completo, alpha)
        if num_pares is not None:
            semestre = i + 1
            resultados[semestre] = num_pares
    return pd.DataFrame(pd.Series(resultados), columns=['NumParesIntegrados'])

In [13]:
# Exemplo de uso:
pasta_dos_csvs = '../cointegration_data/'  # Substitua pelo caminho real da sua pasta
alpha = 0.05

resultado_final_df_corrigido = analisar_pares_integrados_por_semestre_df_corrigido(pasta_dos_csvs, alpha)
print(resultado_final_df_corrigido)

    NumParesIntegrados
1                 8527
2                11456
3                12191
4                 8499
5                 7512
6                 7512
7                 9313
8                 9313
9                 9313
10                9313
11                6882
12                4866
13               11456
14                7814
15                8319
16                7629
17                5787
18                6021
19                5962
20                7739
21                6111
22                4959
23                8158
24               12191
25                8697
26               14305
27                7893
28                7183
29               11327
30                7874
31                7642
32                6985
33                8058
34                6490
35               12191
36                6707
37                5685
38                8356
39                4503
40                6982
41                7276
42                9564
43         